In [2]:
pip install pandas numpy duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import duckdb

Parte I do Projeto  - Investigação
Aqui eu vou fazer uma análise breve do meu arquivo

In [4]:
# Lendo o arquivo CSV
df = pd.read_csv('arquivo_usados/supermarket_sales.csv')

In [5]:
# Estrutura dos dados
print("\n📊 Tipos de dados por coluna:")
dtypes_df = pd.DataFrame({
    'Coluna': df.dtypes.index,
    'Tipo': df.dtypes.values,
    'Valores Únicos': [df[col].nunique() for col in df.columns],
    'Exemplo': [df[col].iloc[0] for col in df.columns]
})
print(dtypes_df.to_string(index=False))



📊 Tipos de dados por coluna:
                 Coluna    Tipo  Valores Únicos           Exemplo
             Invoice ID  object            1000       750-67-8428
                 Branch  object               3                 A
                   City  object               3            Yangon
          Customer type  object               2            Member
                 Gender  object               2            Female
           Product line  object               6 Health and beauty
             Unit price float64             943             74.69
               Quantity   int64              10                 7
                 Tax 5% float64             990           26.1415
                  Total float64             990          548.9715
                   Date  object              89          1/5/2019
                   Time  object             506             13:08
                Payment  object               3           Ewallet
                   cogs float64             99

In [6]:
# Primeiras linhas
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [7]:
# Estatísticas descritivas para colunas numéricas

numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    print(f"\n📈 Análise de '{col}':")
    print(f"   • Mínimo: {df[col].min():.2f}")
    print(f"   • Máximo: {df[col].max():.2f}")
    print(f"   • Média: {df[col].mean():.2f}")
    print(f"   • Mediana: {df[col].median():.2f}")
    print(f"   • Desvio Padrão: {df[col].std():.2f}")
    print(f"   • Q1 (25%): {df[col].quantile(0.25):.2f}")
    print(f"   • Q3 (75%): {df[col].quantile(0.75):.2f}")
    print(f"   • IQR: {df[col].quantile(0.75) - df[col].quantile(0.25):.2f}")


📈 Análise de 'Unit price':
   • Mínimo: 10.08
   • Máximo: 99.96
   • Média: 55.67
   • Mediana: 55.23
   • Desvio Padrão: 26.49
   • Q1 (25%): 32.88
   • Q3 (75%): 77.94
   • IQR: 45.06

📈 Análise de 'Quantity':
   • Mínimo: 1.00
   • Máximo: 10.00
   • Média: 5.51
   • Mediana: 5.00
   • Desvio Padrão: 2.92
   • Q1 (25%): 3.00
   • Q3 (75%): 8.00
   • IQR: 5.00

📈 Análise de 'Tax 5%':
   • Mínimo: 0.51
   • Máximo: 49.65
   • Média: 15.38
   • Mediana: 12.09
   • Desvio Padrão: 11.71
   • Q1 (25%): 5.92
   • Q3 (75%): 22.45
   • IQR: 16.52

📈 Análise de 'Total':
   • Mínimo: 10.68
   • Máximo: 1042.65
   • Média: 322.97
   • Mediana: 253.85
   • Desvio Padrão: 245.89
   • Q1 (25%): 124.42
   • Q3 (75%): 471.35
   • IQR: 346.93

📈 Análise de 'cogs':
   • Mínimo: 10.17
   • Máximo: 993.00
   • Média: 307.59
   • Mediana: 241.76
   • Desvio Padrão: 234.18
   • Q1 (25%): 118.50
   • Q3 (75%): 448.91
   • IQR: 330.41

📈 Análise de 'gross margin percentage':
   • Mínimo: 4.76
   • Máximo:

In [8]:
# Estatísticas descritivas para colunas categóricas
df.describe(include=['object'])

,Invoice ID,Branch,City,Customer type,Gender,Product line,Date,Time,Payment
count,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,1000,3,3,2,2,6,89,506,3
top,750-67-8428,A,Yangon,Member,Female,Fashion accessories,2/7/2019,19:48,Ewallet
freq,1,340,340,501,501,178,20,7,345


In [9]:
# Aqui eu verifico valores ausentes

missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Coluna': missing.index,
    'Valores Ausentes': missing.values,
    'Percentual (%)': missing_pct.values
})

print("\n🔍 Valores ausentes por coluna:")
print(missing_df)

if missing.sum() == 0:
    print("\n✅ Excelente! Nenhum valor ausente encontrado.")
else:
    print(f"\n⚠️  Total de valores ausentes: {missing.sum()}")


🔍 Valores ausentes por coluna:
                     Coluna  Valores Ausentes  Percentual (%)
0                Invoice ID                 0             0.0
1                    Branch                 0             0.0
2                      City                 0             0.0
3             Customer type                 0             0.0
4                    Gender                 0             0.0
5              Product line                 0             0.0
6                Unit price                 0             0.0
7                  Quantity                 0             0.0
8                    Tax 5%                 0             0.0
9                     Total                 0             0.0
10                     Date                 0             0.0
11                     Time                 0             0.0
12                  Payment                 0             0.0
13                     cogs                 0             0.0
14  gross margin percentage           

In [10]:
# Aqui eu verifico duplicatas

duplicates = df.duplicated().sum()
print(f"\n🔍 Total de linhas duplicadas: {duplicates}")

if duplicates > 0:
    print(f"⚠️  {duplicates} duplicatas encontradas ({(duplicates/len(df)*100):.2f}%)")
    print("\nExemplo de duplicatas:")
    print(df[df.duplicated(keep=False)].head())
else:
    print("✅ Nenhuma duplicata encontrada.")



🔍 Total de linhas duplicadas: 0
✅ Nenhuma duplicata encontrada.
